### This is my Proyect of Capstone - Part2 - Add Latitude and Longitude

# We import the librarys

In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup

Get Postal Codes of Canada

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source, 'lxml')

In [4]:
table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
table_rows = table.find_all('tr')

In [6]:
table = []
for ifile in table_rows:
    row = []
    for t in ifile.find_all('td'):
        row =  row + [ t.text.strip() ]
    table.append(row)


table = pd.DataFrame(table, columns=['PostalCode', 'Borough', 'Neighbourhood'])
table.shape

(288, 3)

Remove null date

In [8]:
table = table[~table['PostalCode'].isnull()]
table.shape

(287, 3)

In [9]:
table.head(10)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned
10,M9A,Downtown Toronto,Queen's Park


Remove data in Borough column with value "Not Assigned"

In [10]:
table = table[table['Borough']!="Not assigned"]
table.shape

(210, 3)

Apply groupby functión and then aggregation in neighbourhood

In [11]:
table = table.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(x))
table.reset_index(inplace=True)
table.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Check Neighbourhood with value same to: "Not Assigned"

In [12]:
table[table['Neighbourhood']=="Not assigned"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [13]:
ListIndexNeigh_NotAssigned = table[table['Neighbourhood']=="Not assigned"].index.values.tolist()
ListIndexNeigh_NotAssigned

[85]

In [14]:
for ind in ListIndexNeigh_NotAssigned:
    table.loc[ind]['Neighbourhood'] = table.loc[ind]['Borough']
    print("---")


---


In [15]:
table[table['Neighbourhood']=="Not assigned"]

,PostalCode,Borough,Neighbourhood


In [16]:
table.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [17]:
table.shape

(103, 3)

## Part 2 - Add Latitude and Longitude

Get latitude and longitude of cocl.us

In [18]:
df_cp_coord = pd.read_csv('https://cocl.us/Geospatial_data')
df_cp_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Rename Posta code Column

In [19]:
df_cp_coord.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_cp_coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df = pd.merge(table, df_cp_coord, on='PostalCode')
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
df.shape

(103, 5)

## Part 3: Filtred borough with string Toronto

In [23]:
from geopy.geocoders import Nominatim
import folium

df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


In [24]:
print('The dataframe of Toronto has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe of Toronto has 4 boroughs and 39 neighborhoods.


Get one point media of toronto

In [25]:
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()

print('The geograpical coordinate of Toronto are {}, {}.'.format(lat_toronto, lon_toronto))



The geograpical coordinate of Toronto are 43.66727739999999, -79.39353346923077.


The maps of toronto is

In [26]:
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_toronto)  
    
map_toronto